In [1]:
import requests
import pandas as pd
import numpy as np
import time
import pprint as pp
from tqdm import tqdm
import time
import psutil
import sys

In [2]:
#Prompt User for their Listing Input
inp = input('Enter the name and city of your restaurant separated by a comma:')
name = inp.split(',')[0] #Buona Forchetta
location = inp.split(',')[1] #San Diego

Enter the name and city of your restaurant separated by a comma:Buona Forchetta, San Diego


In [3]:
PARAMETERS = {'location': location, 
              'term': name,
              'limit': 50,
              'offset':0}

In [4]:
#Define the API Key, Endpoint, and Header
API1='ds5abV86lpgzBp767VpgjncvxHLDI64ZqZQABokWL-sRtu6II83zKSLuvhxZNEaHJ_tJ5aUFJRdlIMGnudVKQv61YUkS_vq8AuGeJOz9oPGfyvELw3rDVAVyUWdpXXYx'
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API1}

In [5]:
response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)
if response.status_code!=200:
        print('Error: response status code not 200')
        sys.exit(0)
business_data = response.json()

In [6]:
found = False
businesses = (x for x in business_data['businesses'])
while found == False:
    nxt = next(businesses)
    inp = input("Is this the correct listing?: "+nxt['name']+" (Y/n)")
    if inp.lower() == 'y':
        found = True

Is this the correct listing?: Buona Forchetta (Y/n)y


In [7]:
#nxt at this point is the target listing the user entered

In [8]:
cats = []
for i in range(len(nxt['categories'])):
    cats += [nxt['categories'][i]['title']]

In [9]:
print(cats)
inp = input("What is the index of the correct category?")
cat = cats[int(inp)]

['Pizza', 'Italian']
What is the index of the correct category?1


In [10]:
#Initialize lists for collecting data
name = []
url = []
phone = []
address = []
city = []
state = []
czip = []
rating = []
review_count = []
vertical = []
price = []
latitude = []
longitude = []
yelp_id = []
review1 = []
review2 = []
review3 = []

In [11]:
count = 0

In [12]:
for page in tqdm([0,50,100,150]):
    count+=1
    #try:
    PARAMETERS = {'location': location, 
              'term': cat,
              'limit': 50,
              'offset':page}
    response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)
    if response.status_code!=200:
        print('Error: response status code not 200')
        time.sleep(5)
        sys.exit(0)
    business_data = response.json()
    for business in tqdm(business_data['businesses']):
        name = name + [business['name']]
        url = url + [business['url']]
        address = address + [business['location']['address1']]
        city = city + [business['location']['city']]
        state = state + [business['location']['state']]
        czip = czip + [business['location']['zip_code']]
        phone = phone + [business['phone']]
        latitude = latitude + [business['coordinates']['latitude']]
        longitude = longitude + [business['coordinates']['longitude']]
        try:
            price = price + [business['price']]
        except:
            price = price + ['null']
        yelp_id = yelp_id + [business['id']]
        industries = str()
        for i in business['categories']:
            industries = industries+', '+str(i['title'])
        vertical = vertical + [industries[2:]]
        rating = rating + [business['rating']]
        review_count = review_count + [business['review_count']]
        # Reviews
        response_reviews = requests.get(url = 'https://api.yelp.com/v3/businesses/'+business['id']+'/reviews',
            params = {}, headers = HEADERS)
        review_data = response_reviews.json()
        try:
            review1 += [review_data['reviews'][0]['text']]
        except: review1 += ['null']
        try:
            review2 += [review_data['reviews'][1]['text']]
        except: review2 += ['null']
        try:
            review3 += [review_data['reviews'][2]['text']]
        except: review3 += ['null']
#     except:
#         print('error')

 14%|█▍        | 7/50 [00:03<00:22,  1.92it/s]


100%|██████████| 4/4 [01:52<00:00, 28.05s/it]s]


In [13]:
result = pd.DataFrame({
    'Name':name[:200],
    'URL':url[:200],
    'Phone':phone[:200],
    'Latitude':latitude[:200],
    'Longitude':longitude[:200],
    'Address':address[:200],
    'City':city[:200],
    'State':state[:200],
    'ZIP':czip[:200],
    'Vertical':vertical[:200],
    'Rating':rating[:200],
    'Review Count':review_count[:200],
    'Yelp ID':yelp_id[:200],
    'Review1':review1[:200],
    'Review2':review2[:200],
    'Review3':review3[:200]
})
print('Original length of collected data: '+str(len(result))+' listings.')

Original length of collected data: 200 listings.


In [14]:
def vertical_split(x_):
    first = []
    second = []
    third = []
    for _ in x_:
        x = _.split(', ')
        try:
            first += [x[0]]
        except: first += ['None']
        try:
            second += [x[1]]
        except: second += ['None']
        try:
            third += [x[2]]
        except: third += ['None']
    return first, second, third

In [15]:
result['Vertical1'], result['Vertical2'], result['Vertical3'] = vertical_split(result['Vertical'])

In [16]:
len(result)

200

In [17]:
result.drop_duplicates(subset=['Phone'], inplace=True)
result.drop_duplicates(subset=['Name'], inplace=True)

In [18]:
result['Reviews'] = result['Review1'] + ' ' + result['Review2'] + ' ' + result['Review3']

In [19]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[\d\.]', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('ᐧ', '', text)
    return text

In [20]:
import re
import string
result['Reviews'] = result['Reviews'].apply(lambda x: clean_text_round1(x))

In [21]:
add_stop_words = []

In [22]:
from sklearn.feature_extraction import text

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

In [23]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words, ngram_range=(1,1), tokenizer=LemmaTokenizer())
data_cv = cv.fit_transform(result.Reviews)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = result['Yelp ID']
data_dtm.head()

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,aback,abe,able,abnormal,absolute,absolutely,abundance,accept,accepting,accommodate,...,young,youre,youve,yr,yum,yummmm,yummy,za,zero,zoo
Yelp ID,,,,,,,,,,,,,,,,,,,,,
AlO78to1upUkVhRx12XtVA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aY_z6z6at7wyoW7sFhf4bA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9z2faUubEPjEEpV1zyxH6g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7PGyWA51sSdw8M4fBXa9Gw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
n3sKOKoEk_gKhBvgfIwZaA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Return top popular words 
data_tdm = data_dtm.T
top_dict = {}
for i in data_tdm.columns:
    top = data_tdm[i].sort_values(ascending=False).head(15)
    top_dict[i]= list(zip(top.index, top.values))

top_dict

{'AlO78to1upUkVhRx12XtVA': [('week', 3),
  ('italian', 2),
  ('wa', 2),
  ('grab', 2),
  ('di', 1),
  ('really', 1),
  ('world', 1),
  ('great', 1),
  ('cucina', 1),
  ('multiple', 1),
  ('love', 1),
  ('service', 1),
  ('finish', 1),
  ('pasta', 1),
  ('amazing', 1)],
 'aY_z6z6at7wyoW7sFhf4bA': [('great', 2),
  ('review', 2),
  ('wa', 2),
  ('place', 2),
  ('came', 2),
  ('staff', 1),
  ('pick', 1),
  ('birthday', 1),
  ('friend', 1),
  ('priced', 1),
  ('vacation', 1),
  ('lunch', 1),
  ('said', 1),
  ('salad', 1),
  ('stopped', 1)],
 '9z2faUubEPjEEpV1zyxH6g': [('friend', 2),
  ('bencotto', 2),
  ('zoo', 1),
  ('biophysical', 1),
  ('ive', 1),
  ('sauce', 1),
  ('sausage', 1),
  ('italy', 1),
  ('scientist', 1),
  ('institution', 1),
  ('walked', 1),
  ('creamy', 1),
  ('underbelly', 1),
  ('trip', 1),
  ('located', 1)],
 '7PGyWA51sSdw8M4fBXa9Gw': [('lived', 2),
  ('wa', 2),
  ('year', 2),
  ('pandemic', 2),
  ('takeout', 2),
  ('horrible', 2),
  ('looking', 1),
  ('coming', 1),
  ('

In [26]:
for yid, top_words in top_dict.items():
    print(yid)
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')

AlO78to1upUkVhRx12XtVA
week, italian, wa, grab, di, really, world, great, cucina, multiple, love, service, finish, pasta
---
aY_z6z6at7wyoW7sFhf4bA
great, review, wa, place, came, staff, pick, birthday, friend, priced, vacation, lunch, said, salad
---
9z2faUubEPjEEpV1zyxH6g
friend, bencotto, zoo, biophysical, ive, sauce, sausage, italy, scientist, institution, walked, creamy, underbelly, trip
---
7PGyWA51sSdw8M4fBXa9Gw
lived, wa, year, pandemic, takeout, horrible, looking, coming, great, place, sizing, people, embarcadero, ha
---
n3sKOKoEk_gKhBvgfIwZaA
wa, meal, pandemic, pizza, close, ordering, sat, soup, pasta, treat, including, pretty, lockdown, buono
---
W0dzJ7E6R3kyAOXIlO_tbw
takeaway, o, like, wind, t, caesar, c, ha, time, tiny, building, nestled, n, helpful
---
4GT1sQ_pcKk4gBMrJddexw
food, area, delivery, college, italian, order, delicious, live, wrong, pandemic, pizza, ultimate, drive, fails
---
9QifYbaGw27jgDBhXjg1vQ
u, great, wa, arrived, recommend, annoying, checked, place, 

In [27]:
# Look at the most common top words --> add them to the stop word list
from collections import Counter

# Let's first pull out the top 30 words for each comedian
words = []
for yid in top_dict.keys():
    top = [word for (word, count) in top_dict[yid]]
    for t in top:
        words.append(t)
        
words

['week',
 'italian',
 'wa',
 'grab',
 'di',
 'really',
 'world',
 'great',
 'cucina',
 'multiple',
 'love',
 'service',
 'finish',
 'pasta',
 'amazing',
 'great',
 'review',
 'wa',
 'place',
 'came',
 'staff',
 'pick',
 'birthday',
 'friend',
 'priced',
 'vacation',
 'lunch',
 'said',
 'salad',
 'stopped',
 'friend',
 'bencotto',
 'zoo',
 'biophysical',
 'ive',
 'sauce',
 'sausage',
 'italy',
 'scientist',
 'institution',
 'walked',
 'creamy',
 'underbelly',
 'trip',
 'located',
 'lived',
 'wa',
 'year',
 'pandemic',
 'takeout',
 'horrible',
 'looking',
 'coming',
 'great',
 'place',
 'sizing',
 'people',
 'embarcadero',
 'ha',
 'comfort',
 'wa',
 'meal',
 'pandemic',
 'pizza',
 'close',
 'ordering',
 'sat',
 'soup',
 'pasta',
 'treat',
 'including',
 'pretty',
 'lockdown',
 'buono',
 'friday',
 'takeaway',
 'o',
 'like',
 'wind',
 't',
 'caesar',
 'c',
 'ha',
 'time',
 'tiny',
 'building',
 'nestled',
 'n',
 'helpful',
 'bolognese',
 'food',
 'area',
 'delivery',
 'college',
 'italian

In [28]:
Counter(words).most_common()

[('wa', 108),
 ('pizza', 62),
 ('place', 59),
 ('great', 48),
 ('food', 47),
 ('time', 42),
 ('service', 35),
 ('delicious', 33),
 ('italian', 32),
 ('love', 29),
 ('just', 27),
 ('pasta', 26),
 ('ive', 26),
 ('came', 24),
 ('diego', 24),
 ('amazing', 21),
 ('san', 21),
 ('restaurant', 21),
 ('menu', 20),
 ('try', 18),
 ('ha', 17),
 ('like', 17),
 ('really', 16),
 ('sauce', 16),
 ('order', 16),
 ('went', 15),
 ('delivery', 14),
 ('night', 14),
 ('ordered', 14),
 ('salad', 13),
 ('excellent', 13),
 ('local', 13),
 ('best', 13),
 ('fresh', 12),
 ('good', 12),
 ('long', 12),
 ('work', 12),
 ('friend', 11),
 ('pandemic', 11),
 ('friendly', 11),
 ('thank', 11),
 ('people', 10),
 ('say', 10),
 ('called', 10),
 ('family', 10),
 ('kind', 10),
 ('business', 10),
 ('lunch', 9),
 ('takeout', 9),
 ('looking', 9),
 ('meal', 9),
 ('pretty', 9),
 ('new', 9),
 ('im', 9),
 ('day', 9),
 ('customer', 9),
 ('know', 9),
 ('did', 9),
 ('happy', 9),
 ('star', 9),
 ('thing', 9),
 ('italy', 8),
 ('u', 8),
 ('e

In [29]:
add_stop_words += [word for word, count in Counter(words).most_common() if count > len(top_dict)/3]
add_stop_words

['wa']

In [30]:
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

cv = CountVectorizer(stop_words=stop_words, ngram_range=(1,1), tokenizer=LemmaTokenizer())
data_cv = cv.fit_transform(result.Reviews)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = result['Yelp ID']
data_dtm.head()

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,aback,abe,able,abnormal,absolute,absolutely,abundance,accept,accepting,accommodate,...,young,youre,youve,yr,yum,yummmm,yummy,za,zero,zoo
Yelp ID,,,,,,,,,,,,,,,,,,,,,
AlO78to1upUkVhRx12XtVA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aY_z6z6at7wyoW7sFhf4bA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9z2faUubEPjEEpV1zyxH6g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7PGyWA51sSdw8M4fBXa9Gw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
n3sKOKoEk_gKhBvgfIwZaA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# SENTIMENT ANALYSIS

In [32]:
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

In [33]:
result['polarity'] = result['Reviews'].apply(pol)
result['subjectivity'] = result['Reviews'].apply(sub)

In [34]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]

for index, yid in enumerate(result.index):
    x = result.polarity.loc[yid]
    y = result.subjectivity.loc[yid]
    plt.scatter(x, y, color='blue')
    plt.text(x+.001, y+.001, result[yid][index], fontsize=10)
    plt.xlim(-.01, .25) 
    
plt.title('Sentiment Analysis', fontsize=20)
plt.xlabel('<-- Negative -------- Positive -->', fontsize=15)
plt.ylabel('<-- Facts -------- Opinions -->', fontsize=15)

plt.show()

KeyError: 0

In [ ]:
word1 = []
word2 = []
word3 = []
word4 = []
word5 = []

for i in range(len(data_dtm)):
    temp = data_dtm.iloc[i].sort_values(ascending=False)[:5].index
    word1 += [temp[0]]
    word2 += [temp[1]]
    word3 += [temp[2]]
    word4 += [temp[3]]
    word5 += [temp[4]]

In [ ]:
result['Word1'] = word1
result['Word2'] = word2
result['Word3'] = word3
result['Word4'] = word4
result['Word5'] = word5

In [ ]:
result.head()

In [ ]:
result

In [ ]:
import collections
words = list(result['Word1']) + list(result['Word2']) + list(result['Word3']) + list(result['Word4']) + list(result['Word5'])

counter=collections.Counter(words)

In [ ]:
counter

In [ ]:
counter.most_common()

In [ ]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(' '.join(words))
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
temp = result.copy(deep=True)
temp['color'] = [0]*len(temp)
temp.iloc[2,27] = 1

In [ ]:
import plotly.express as px

fig = px.scatter(temp, x="polarity", y="subjectivity", color="color", hover_data=['Name'])
fig.show()

In [ ]:
df = pd.DataFrame({'words':list(counter.keys()),
             'count':list(counter.values())}).sort_values('count', ascending=False)

In [ ]:
import plotly.express as px

fig = px.bar(df, y='count', x='words', text='count')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(uniformtext_minsize=5, uniformtext_mode='hide')
fig.show()

In [ ]:
df = result[result['Name']!=nxt['name']]
target = result[result['Name']==nxt['name']]

In [ ]:
target_words = ['pizza','pandemic','close','meal','evening']
target

In [ ]:
t1 = df[df['Word1'].apply(lambda x: True if x in target_words else False)]
t2 = df[df['Word2'].apply(lambda x: True if x in target_words else False)]
t3 = df[df['Word3'].apply(lambda x: True if x in target_words else False)]
t4 = df[df['Word4'].apply(lambda x: True if x in target_words else False)]
t5 = df[df['Word5'].apply(lambda x: True if x in target_words else False)]

concat = pd.concat([t1,t2,t3,t4,t5])

In [ ]:
concat = concat.drop_duplicates('Name')
print(f'There are {len(concat)} out of {len(df)} listings that have any of the same top words as {nxt["name"]}')

In [ ]:
t1 = df[df['Word1'].apply(lambda x: True if x in target_words else False)]
t2 = df[df['Word2'].apply(lambda x: True if x in target_words else False)]
t3 = df[df['Word3'].apply(lambda x: True if x in target_words else False)]
t4 = df[df['Word4'].apply(lambda x: True if x in target_words else False)]
t5 = df[df['Word5'].apply(lambda x: True if x in target_words else False)]

concat = pd.concat([t1,t2,t3,t4,t5])
concat['count'] = [1]*len(concat)

In [ ]:
t4

In [107]:
concat.groupby('Name').count()

,URL,Phone,Latitude,Longitude,Address,City,State,ZIP,Vertical,Rating,...,Vertical3,Reviews,polarity,subjectivity,Word1,Word2,Word3,Word4,Word5,count
Name,,,,,,,,,,,,,,,,,,,,,
Alfredo's Pizzeria,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Alfredo’s Pizzeria,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Ambrogio15,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Barbusa,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Bronx Pizza,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Brothers' Giant Pizza,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Buon Appetito,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Caffè Calabria,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Catalano's Pizzeria,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## LDA

In [53]:
result

,Name,URL,Phone,Latitude,Longitude,Address,City,State,ZIP,Vertical,Rating,Review Count,Yelp ID,Review1,Review2,Review3,Vertical1,Vertical2,Vertical3,Reviews
0,Buona Forchetta,https://www.yelp.com/biz/buona-forchetta-san-d...,+16193814844,32.721302,-117.130202,3001 Beech St,San Diego,CA,92102,"Pizza, Italian",4.5,2421,n3sKOKoEk_gKhBvgfIwZaA,"LOCKDOWN HOURS:\nMon-Thurs 12-3pm, 4-8:30pm\nS...",Covid pandemic take out option with buono was ...,Enjoyed a lovely pandemic take-out meal on a F...,Pizza,Italian,None,"LOCKDOWN HOURS:\nMon-Thurs 12-3pm, 4-8:30pm\nS..."
1,Buon Appetito,https://www.yelp.com/biz/buon-appetito-san-die...,+16192389880,32.722356,-117.168111,1609 India St,San Diego,CA,92101,Italian,4.0,2296,7PGyWA51sSdw8M4fBXa9Gw,Can't say enough about this place! Lived in th...,We ordered take-out last week and it was pheno...,My husband and I have lived in Little Italy fo...,Italian,None,None,Can't say enough about this place! Lived in th...
2,Bencotto Italian Kitchen,https://www.yelp.com/biz/bencotto-italian-kitc...,+16194504786,32.724449,-117.168975,750 W Fir St,San Diego,CA,92101,"Italian, Bars, Desserts",4.0,2505,9z2faUubEPjEEpV1zyxH6g,I have been coming to Bencotto for at least 5 ...,Came here after a trip to the zoo with my frie...,"Bencotto, an institution located in the heart ...",Italian,Bars,Desserts,I have been coming to Bencotto for at least 5 ...
3,Cucina Basilico,https://www.yelp.com/biz/cucina-basilico-san-d...,+18588746436,32.810650,-117.116113,3755 Murphy Canyon Rd,San Diego,CA,92123,Italian,4.5,574,AlO78to1upUkVhRx12XtVA,"During this ""stay at home"" order, we've been o...",My friends and I decided to go in for dinner l...,AMAZING Homemade Italian Food!\n\nI come here ...,Italian,None,None,"During this ""stay at home"" order, we've been o..."
4,Ristorante Illando,https://www.yelp.com/biz/ristorante-illando-sa...,+16196935204,32.724049,-117.168190,1825 India Street,San Diego,CA,92101,Italian,4.5,973,W0dzJ7E6R3kyAOXIlO_tbw,Ordered takeaway for Easter meal during time o...,Wait staff 5. Very attentive. Very helpful \nS...,= L O C A T I O N =\nLike other reviewers have...,Italian,None,None,Ordered takeaway for Easter meal during time o...
5,Pesto Italian Craft Kitchen,https://www.yelp.com/biz/pesto-italian-craft-k...,+16192652700,32.760617,-117.068064,6011 El Cajon Blvd,San Diego,CA,92115,"Pizza, Italian",4.5,1223,4GT1sQ_pcKk4gBMrJddexw,"The best Italian food, pizza in College area. ...",Huge shout out to the team at Pesto Italian Cr...,Amazing service! \n\nFood taste great! Amazing...,Pizza,Italian,None,"The best Italian food, pizza in College area. ..."
6,Filippi's Pizza Grotto Little Italy,https://www.yelp.com/biz/filippis-pizza-grotto...,+16192325094,32.723750,-117.168130,1747 India St,San Diego,CA,92101,"Italian, Pizza, Bars",4.0,2825,PzGU5brzhsVqumdU4qE5Rw,"Been here so many times, time for a local revi...",Came here to shop during the pandemic in the m...,I'm not a pizza person but when my husband ask...,Italian,Pizza,Bars,"Been here so many times, time for a local revi..."
7,Stella Italianfare UTC,https://www.yelp.com/biz/stella-italianfare-ut...,+18586221202,32.867847,-117.206213,8935 Towne Centre Dr,San Diego,CA,92122,"Italian, Pizza",4.5,457,hSoOhMcv6QAlr7zGhza3iQ,"I've been there many times, it became my favor...",Eaten here plenty of times over years and have...,I have to say that Stella Italianfare's servic...,Italian,Pizza,None,"I've been there many times, it became my favor..."
8,Barbusa,https://www.yelp.com/biz/barbusa-san-diego?adj...,+16192381917,32.724807,-117.168491,1917 India St,San Diego,CA,92101,"Sicilian, Seafood, Pizza",4.0,2006,0t0AfOKctJebkYb89G1vdg,**REVIEW BASED ON PRE PANDEMIC EXPERIENCE**\n\...,In the days before the Pandemic and the quaran...,This is the best Italian restaurant I've ever ...,Sicilian,Seafood,Pizza,**REVIEW BASED ON PRE PANDEMIC EXPERIENCE**\n\...
9,Pomodoro Ristorante Italiano,https://www.yelp.com/biz/pomodoro-ristorante-i...,+16195231301,32.721122,-117.230367,28

In [63]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

from nltk import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danielhartmann/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [65]:
doc_sample = result.iloc[0]['Reviews'].replace('\n',' ')

In [66]:
words = []
print('original document: ')
for word in doc_sample.split(' '):
    words.append(word)
    
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['LOCKDOWN', 'HOURS:', 'Mon-Thurs', '12-3pm,', '4-8:30pm', 'Sat', 'closes', 'at', '9pm', 'Sun', 'closes', 'at', '8pm', '', 'This', 'is', 'the', 'best', 'pizza', 'in', 'town', '(or', 'at', 'least', 'top', 'three,', 'including', "Friendly's...", 'Covid', 'pandemic', 'take', 'out', 'option', 'with', 'buono', 'was', 'pretty', 'neat', '!', '', 'Ordering', 'system', 'was', 'quick', 'over', 'the', 'phone-they', 'two', 'numbers', '(check', 'out', 'photo', 'for', 'the', 'second...', 'Enjoyed', 'a', 'lovely', 'pandemic', 'take-out', 'meal', 'on', 'a', 'Friday', 'evening!', 'Purchased', 'the', 'family', 'meal,', 'which', 'was', '$39', 'for', 'two', 'pizzas,', 'pasta', 'and', 'soup.', 'It', 'was', 'such', 'a', 'treat....']


 tokenized and lemmatized document: 
['lockdown', 'hour', 'thur', 'close', 'close', 'best', 'pizza', 'town', 'includ', 'friendli', 'covid', 'pandem', 'option', 'buono', 'pretti', 'neat', 'order', 'quick', 'phone', 'number', 'check', 'photo', 'second', 'enjo

In [68]:
processed_docs = result['Reviews'].replace('\n',' ').map(preprocess)
processed_docs[:10]

0    [lockdown, hour, thur, close, close, best, piz...
1    [place, live, embarcadero, year, horribl, pand...
2    [come, bencotto, year, night, organ, dinner, s...
3    [stay, home, order, order, takeout, multipl, t...
4    [order, takeaway, easter, meal, time, covid, q...
5    [best, italian, food, pizza, colleg, area, liv...
6    [time, time, local, review, normal, split, mea...
7    [time, favorit, italian, restaur, jolla, best,...
8    [review, base, pandem, experi, serious, best, ...
9    [place, soooo, good, amaz, servic, hard, figur...
Name: Reviews, dtype: object

In [69]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 best
1 buono
2 check
3 close
4 covid
5 enjoy
6 even
7 famili
8 friday
9 friendli
10 hour


In [102]:
dictionary.filter_extremes(no_below=1, no_above=0.33, keep_n=100000)

In [103]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[-1]

[(0, 1), (4, 1), (5, 1), (11, 1), (39, 1), (52, 1), (58, 1), (60, 1)]

In [73]:
bow_doc_4310 = bow_corpus[-1]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 0 ("best") appears 1 time.
Word 4 ("famili") appears 1 time.
Word 5 ("friendli") appears 1 time.
Word 7 ("order") appears 1 time.
Word 9 ("pizza") appears 5 time.
Word 13 ("good") appears 1 time.
Word 44 ("deliveri") appears 1 time.
Word 57 ("excel") appears 1 time.
Word 63 ("thank") appears 1 time.
Word 65 ("delici") appears 1 time.


In [74]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.20689914436476317),
 (1, 0.650119369020075),
 (2, 0.23567468648914108),
 (3, 0.2675086002612818),
 (4, 0.24399727041640049),
 (5, 0.2839933901874055),
 (6, 0.19171039844714124),
 (7, 0.10376625668283877),
 (8, 0.22048594057151913),
 (9, 0.24488594811245792),
 (10, 0.33338226843729696)]


In [108]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [109]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.047*"year" + 0.045*"come" + 0.043*"great" + 0.039*"best" + 0.038*"favorit" + 0.033*"love" + 0.032*"review" + 0.031*"diego" + 0.031*"italian" + 0.029*"delici"
Topic: 1 
Words: 0.074*"good" + 0.059*"pasta" + 0.045*"italian" + 0.044*"come" + 0.038*"night" + 0.037*"amaz" + 0.035*"delici" + 0.028*"year" + 0.028*"go" + 0.026*"sauc"
Topic: 2 
Words: 0.051*"love" + 0.050*"open" + 0.046*"littl" + 0.038*"menu" + 0.036*"come" + 0.035*"dinner" + 0.030*"night" + 0.030*"takeout" + 0.028*"wonder" + 0.028*"italian"
Topic: 3 
Words: 0.061*"servic" + 0.048*"love" + 0.043*"amaz" + 0.042*"famili" + 0.040*"good" + 0.039*"come" + 0.038*"excel" + 0.033*"go" + 0.032*"friendli" + 0.028*"nice"
Topic: 4 
Words: 0.058*"deliveri" + 0.055*"italian" + 0.040*"great" + 0.036*"best" + 0.035*"delici" + 0.034*"servic" + 0.034*"amaz" + 0.025*"open" + 0.025*"local" + 0.024*"look"


In [111]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=1)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Process ForkPoolWorker-18:
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "//anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "//anaconda3/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamulticore.py", line 337, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py", line 742, in do_estep
    gamma, sstats = self.inference(chunk, collect_sstats=True)
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py", line 680, in inference
    expElogbetad = self.expElogbeta[:, ids]
IndexError: index 71 is out of bounds for axis 1 with size 71
Process ForkPoolWorker-19:
Traceback (most recent call last):
  File "//an

KeyboardInterrupt: 

  File "//anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "//anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [135]:
lda_model.print_topic(0)

'0.047*"year" + 0.045*"come" + 0.043*"great" + 0.039*"best" + 0.038*"favorit" + 0.033*"love" + 0.032*"review" + 0.031*"diego" + 0.031*"italian" + 0.029*"delici"'

In [142]:
len(result)

199

In [148]:
bow_vector = dictionary.doc2bow(preprocess(result['Reviews'][0]))

'0.058*"deliveri" + 0.055*"italian" + 0.040*"great" + 0.036*"best" + 0.035*"delici"'

In [160]:
import re

word1 = []
word2 = []
word3 = []
word4 = []
word5 = []

for i in result['Reviews']:
    bow_vector = dictionary.doc2bow(preprocess(i))
    topics = lda_model.print_topic(sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1])[0][0],5)
    words = re.findall(r'"(.*?)"', topics)
    word1 += [words[0]]
    word2 += [words[1]]
    word3 += [words[2]]
    word4 += [words[3]]
    word5 += [words[4]]

In [163]:
result['Word1'] = word1
result['Word2'] = word2
result['Word3'] = word3
result['Word4'] = word4
result['Word5'] = word5

In [167]:
result.head()

,Name,URL,Phone,Latitude,Longitude,Address,City,State,ZIP,Vertical,...,Review3,Vertical1,Vertical2,Vertical3,Reviews,Word1,Word2,Word3,Word4,Word5
0,Buona Forchetta,https://www.yelp.com/biz/buona-forchetta-san-d...,+16193814844,32.721302,-117.130202,3001 Beech St,San Diego,CA,92102,"Pizza, Italian",...,Enjoyed a lovely pandemic take-out meal on a F...,Pizza,Italian,None,"LOCKDOWN HOURS:\nMon-Thurs 12-3pm, 4-8:30pm\nS...",deliveri,italian,great,best,delici
1,Buon Appetito,https://www.yelp.com/biz/buon-appetito-san-die...,+16192389880,32.722356,-117.168111,1609 India St,San Diego,CA,92101,Italian,...,My husband and I have lived in Little Italy fo...,Italian,None,None,Can't say enough about this place! Lived in th...,year,come,great,best,favorit
2,Bencotto Italian Kitchen,https://www.yelp.com/biz/bencotto-italian-kitc...,+16194504786,32.724449,-117.168975,750 W Fir St,San Diego,CA,92101,"Italian, Bars, Desserts",...,"Bencotto, an institution located in the heart ...",Italian,Bars,Desserts,I have been coming to Bencotto for at least 5 ...,year,come,great,best,favorit
3,Cucina Basilico,https://www.yelp.com/biz/cucina-basilico-san-d...,+18588746436,32.810650,-117.116113,3755 Murphy Canyon Rd,San Diego,CA,92123,Italian,...,AMAZING Homemade Italian Food!\n\nI come here ...,Italian,None,None,"During this ""stay at home"" order, we've been o...",deliveri,italian,great,best,delici
4,Ristorante Illando,https://www.yelp.com/biz/ristorante-illando-sa...,+16196935204,32.724049,-117.168190,1825 India Street,San Diego,CA,92101,Italian,...,= L O C A T I O N =\nLike other reviewers have...,Italian,None,None,Ordered takeaway for Easter meal during time o...,year,come,great,best,favorit


In [166]:
result.iloc[1]['Reviews']

"Can't say enough about this place! Lived in the Embarcadero for many years and during this horrible pandemic felt so much comfort in ordering take away... We ordered take-out last week and it was phenomenal! The portion sizing was great for two people. Definitely recommend for anyone looking for good takeout... My husband and I have lived in Little Italy for 14 years and have been coming here since Day 1.  During this horrible pandemic, nothing has changed when it..."

In [168]:
result.iloc[2]['Reviews']

"I have been coming to Bencotto for at least 5 years now. Last night, I organized a dinner for ten for my scientist and coworkers here for biophysical... Came here after a trip to the zoo with my friends! I got their pappardelle with a creamy white sauce and sausage! I can't remember the name of the dish but... Bencotto, an institution located in the heart of Little Italy next to Underbelly. I've heard so many things about this restaurant from friends and walked..."

In [170]:
import collections
words = list(result['Word1']) + list(result['Word2']) + list(result['Word3']) + list(result['Word4']) + list(result['Word5'])

counter=collections.Counter(words)

In [173]:
counter

Counter({'deliveri': 40,
         'year': 68,
         'good': 66,
         'love': 47,
         'servic': 22,
         'italian': 84,
         'come': 137,
         'pasta': 44,
         'open': 25,
         'great': 108,
         'littl': 25,
         'amaz': 22,
         'best': 108,
         'menu': 25,
         'famili': 22,
         'delici': 40,
         'favorit': 68,
         'night': 44})

In [169]:
counter

Counter({'pizza': 46,
         'pandemic': 4,
         'bencotto': 1,
         'week': 2,
         'o': 1,
         'food': 38,
         'marinara': 1,
         'time': 16,
         'amazing': 10,
         'extremely': 3,
         'order': 12,
         'office': 1,
         'u': 4,
         'incredible': 1,
         'nice': 1,
         'godfather': 1,
         'night': 9,
         'null': 8,
         'open': 5,
         'new': 5,
         'place': 26,
         'delivery': 5,
         'careful': 2,
         'hospital': 1,
         'restaurant': 28,
         'little': 6,
         'valentine': 2,
         'dinner': 9,
         'gem': 1,
         'vegan': 1,
         'italian': 15,
         'ha': 3,
         'mom': 1,
         'sauce': 3,
         'coffee': 2,
         'wonderful': 5,
         'favorite': 6,
         'able': 1,
         'local': 4,
         'kitchen': 1,
         'service': 11,
         'nonna': 1,
         'reservation': 2,
         'menu': 3,
         'people': 2,
      

In [94]:
result = result.merge(data_dtm.reset_index(), on='Yelp ID', how='left')

In [95]:
result.to_csv('result.csv')

In [96]:
result

,Name,URL,Phone,Latitude,Longitude,Address,City,State,ZIP,Vertical,...,yummy alfredo,yummy fresh,yummy usually,za,za ive,zero,zero flavor,zoo,zoo feeling,zoo friend
0,Cucina Basilico,https://www.yelp.com/biz/cucina-basilico-san-d...,+18588746436,32.810650,-117.116113,3755 Murphy Canyon Rd,San Diego,CA,92123,Italian,...,0,0,0,0,0,0,0,0,0,0
1,Buon Appetito,https://www.yelp.com/biz/buon-appetito-san-die...,+16192389880,32.722356,-117.168111,1609 India St,San Diego,CA,92101,Italian,...,0,0,0,0,0,0,0,0,0,0
2,Buona Forchetta,https://www.yelp.com/biz/buona-forchetta-san-d...,+16193814844,32.721302,-117.130202,3001 Beech St,San Diego,CA,92102,"Pizza, Italian",...,0,0,0,0,0,0,0,0,0,0
3,Pesto Italian Craft Kitchen,https://www.yelp.com/biz/pesto-italian-craft-k...,+16192652700,32.760617,-117.068064,6011 El Cajon Blvd,San Diego,CA,92115,"Pizza, Italian",...,0,0,0,0,0,0,0,0,0,0
4,Filippi's Pizza Grotto Little Italy,https://www.yelp.com/biz/filippis-pizza-grotto...,+16192325094,32.723750,-117.168130,1747 India St,San Diego,CA,92101,"Italian, Pizza, Bars",...,0,0,0,0,0,0,0,0,0,0
5,CUCINA urbana,https://www.yelp.com/biz/cucina-urbana-san-die...,+16192392222,32.731299,-117.160156,505 Laurel St,San Diego,CA,92101,"Italian, Pizza, Wine Bars",...,0,0,0,0,0,0,0,0,0,0
6,Bencotto Italian Kitchen,https://www.yelp.com/biz/bencotto-italian-kitc...,+16194504786,32.724449,-117.168975,750 W Fir St,San Diego,CA,92101,"Italian, Bars, Desserts",...,0,0,0,0,0,0,0,1,0,1
7,The Godfather Restaurant,https://www.yelp.com/biz/the-godfather-restaur...,+18585601747,32.833260,-117.151680,7878 Clairemont Mesa Blvd,San Diego,CA,92111,"Italian, Venues & Event Spaces, Wine Bars",...,0,0,0,0,0,0,0,0,0,0
8,Ristorante Illando,https://www.yelp.com/biz/ristorante-illando-sa...,+16196935204,32.724049,-117.168190,1825 India Street,San Diego,CA,92101,Italian,...,0,0,0,0,0,0,0,0,0,0
9,Pomodoro Ristorante Italiano,https://www.yelp.com/biz/pomodoro-ristorante-i...,+16195231301,32.721122,-117.230367,2833 Avenida De Portugal,San Diego,CA,92106,Italian,...,0,0,0,0,0,0,0,0,0,0


In [101]:
dictionary = gensim.corpora.Dictionary(result['Reviews'].apply(lambda x: x.split(' ')))

In [102]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 a
1 always
2 amazing
3 and
4 at
5 barbabietola
6 basilico
7 been
8 come
9 coming
10 cucina


In [103]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [106]:
bow_corpus = [dictionary.doc2bow(doc) for doc in result['Reviews'].apply(lambda x: x.split(' '))]
bow_corpus[-1]

[(10, 1),
 (11, 1),
 (12, 1),
 (19, 1),
 (20, 2),
 (28, 1),
 (36, 1),
 (38, 1),
 (39, 1),
 (42, 1),
 (59, 1),
 (67, 1),
 (71, 1),
 (87, 2),
 (96, 1),
 (106, 2),
 (114, 1),
 (119, 1),
 (135, 1),
 (136, 1)]

In [107]:
from gensim import corpora, models
import gensim

lda_model = gensim.models.LdaMulticore(result['Reviews'], num_topics=10, id2word=dictionary, passes=2, workers=2)

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "//anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "//anaconda3/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamulticore.py", line 337, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py", line 742, in do_estep
    gamma, sstats = self.inference(chunk, collect_sstats=True)
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py", line 673, in inference
    ids = [int(idx) for idx, _ in doc]
  File "//anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py", line 673, in <listcomp>
    ids = [int(idx) for idx, _ in doc]
Val

KeyboardInterrupt: 

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
result = pd.get_dummies(result, columns=['City','ZIP','Vertical1','Vertical2','Vertical3'], drop_first=True)

In [8]:
result['Ranking'] = list(range(1,len(result)+1))

In [ ]:
result_ = result[['Name','URL','Phone','Address','State','Vertical','Yelp ID']]
result = result.drop(['Name','URL','Phone','Address','State','Vertical','Yelp ID'],axis=1)

In [26]:
result.head()

,Rating,Review Count,Review1,Review2,Review3,City_San Diego,ZIP_92103,ZIP_92104,ZIP_92106,ZIP_92107,...,Vertical3_Soup,Vertical3_Sports Bars,Vertical3_Spray Tanning,Vertical3_Summer Camps,Vertical3_Tires,Vertical3_Venues & Event Spaces,Vertical3_Waxing,Vertical3_Wine & Spirits,Vertical3_Yoga,Ranking
0,4.0,665,FREE parking and admission for amusement young...,While my young kids really enjoyed the experie...,"If you're visiting San Diego or Mission Beach,...",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,5.0,900,Best place in San Diego. I always take my fami...,"Sunset Cliffs is so beautiful, probably the be...",Intense but pretty cool. People can be obnoxio...,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
2,4.5,262,null,null,null,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,4.5,166,This was our first touristy stop in San Diego ...,Well Just got time to review this cool Attract...,Awesome and totally unexpected! The prices her...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,4.0,207,We are lucky to live here in San Diego.\n\nObv...,We need a more aggressive and proactive respon...,One of the finest cities in all the world. Wh...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [27]:
import matplotlib.pyplot as plt  

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = result[['Rating','Review Count']]#.drop(['Ranking','Review1','Review2','Review3'],axis=1)
y = result.Ranking
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
linreg = LogisticRegression().fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [11]:
#To retrieve the intercept:
print(linreg.intercept_)
#For retrieving the slope:
print(linreg.coef_)

[-4.11036843e-04  1.29248654e-03 -3.01678780e-03 -2.30595828e-04
  6.69897441e-04  8.01375700e-04 -1.28748584e-04 -3.50065674e-05
 -1.19177053e-03 -4.47895212e-04  3.92142340e-04  7.52919134e-04
 -2.49148965e-03  1.02410144e-03  9.01456327e-04  6.56602330e-04
 -3.78706122e-05 -3.64573719e-04 -1.21915207e-03 -5.44482905e-04
 -5.88703314e-04  3.59462166e-04  7.83903541e-04 -9.05992416e-04
 -3.76948652e-04  4.26520725e-06  1.31110171e-03  9.44220909e-04
  1.19461641e-03  6.41583110e-04  1.82438010e-04  3.42110940e-04
 -7.45117661e-04  6.95086352e-04 -8.09322805e-04 -1.86685468e-05
  1.05880941e-03 -9.78173549e-05 -1.85024573e-03  1.31757613e-03
  4.96869399e-04  5.84238147e-04  1.65974782e-05 -4.49376010e-04
 -7.54810396e-04  1.28071790e-03  4.19989222e-04  1.31347150e-03
 -5.30986862e-04 -1.84423157e-03 -4.50855641e-04  6.10893244e-04
 -1.31883369e-03  1.31530920e-03  8.16370476e-04  4.14816647e-04
  3.04794927e-04 -3.31356230e-05  1.14722622e-04  5.93128180e-04
  1.15700543e-03  1.05821

In [12]:
y_pred = linreg.predict(X_test)

In [13]:
df = pd.DataFrame({'Actual': np.array(y_test).flatten(), 'Predicted': y_pred.flatten()})
df.head()

,Actual,Predicted
0,79,101
1,156,101
2,129,101
3,56,4
4,95,101


In [14]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 58.95
Mean Squared Error: 5208.25
Root Mean Squared Error: 72.16820629612461
